# Fault Diagnosis Toolbox in Python
## - Code Generation

Erik Frisk<br>
<frisk@isy.liu.se><br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

## Basic Python imports

In [2]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import sys
import numpy as np
import sympy as sym
import scipy.sparse as sp

new_paths = ['../Models/', '../Misc/', 'faultdiagnosistoolbox/build/lib.macosx-10.6-x86_64-2.7/']
[sys.path.append(d) for d in new_paths if not d in sys.path];
from misc import *
import faultdiagnosistoolbox as fdt
import faultdiagnosistoolbox.CodeGeneration as codegen

## Define a diagnosis models -- structural Three Tank model

In [3]:
#from ThreeTank_model import model
from ThreeTank_model_sym import model

In [10]:
print "Model: %s" % model.name
print "  (ne, nx, nf, nz) = (%d,%d,%d,%d)" % (model.ne(), model.nx(), model.nf(), model.nz())
print "  sprank(X) = %d" % model.sprank()
if model.IsDynamic():
    print "  Dynamic model"
else:
        print "  Static model"

Model: Three Tank System
  (ne, nx, nf, nz) = (12,10,6,3)
  sprank(X) = 10
  Dynamic model


In [11]:
print "Searching for MSO sets..."
msos=model.MSO()
print "Found %d mso sets" % len(msos)
print msos

Searching for MSO sets...
Found 6 mso sets
[array([ 7,  6,  4, 10,  3,  8,  9,  1,  2,  5, 11], dtype=int32)
 array([ 7,  6,  4, 10,  3,  8,  9,  0], dtype=int32)
 array([ 7,  6,  4, 10,  1,  2,  5, 11,  0], dtype=int32)
 array([ 7,  6,  3,  8,  9,  1,  2,  5, 11,  0], dtype=int32)
 array([ 7,  4, 10,  3,  8,  9,  1,  2,  5, 11,  0], dtype=int32)
 array([ 6,  4, 10,  3,  8,  9,  1,  2,  5, 11,  0], dtype=int32)]


In [12]:
m=2
mso = msos[m]
rIdx = model.MSOCausalitySweep(mso,causality='int') # Boolean index array to integral causality redundant equation
if np.any(rIdx):
    red = mso[rIdx][0] # Take the first one
    m0 = [e for e in mso if e!=red]
    Gamma = model.Matching(m0)
    print "Found integral causality matching (mso " + str(m) + ", red = " + str(red) + ")"
else:
    print "No integral causality matching exists"

Found integral causality matching (mso 2, red = 7)


In [13]:
#codegen.SeqResGen(model, Gamma, red, 'ResGen_3_8', language='Python')
model.SeqResGen(Gamma, red, 'ResGen_3_8', language='Python')

Generating residual generator ResGen_3_8 (Python)
  Generating code for the exactly determined part: ..
  Generating code for the residual equations
  Writing residual generator file
File ResGen_3_8.py generated.


In [14]:
import ResGen_3_8
reload(ResGen_3_8)
help(ResGen_3_8.ResGen_3_8)

Help on function ResGen_3_8 in module ResGen_3_8:

ResGen_3_8(z, state, params, Ts)
    RESGEN_3_8 Sequential residual generator for model 'Three Tank System'
    Causality: int
    
    Structurally sensitive to faults: fV1, fV2, fV3, fT2, fT3
    
    Example of basic usage:
    Let z be the observations, then the residual generator can be simulated by:
    
    for zk in z:
        rk, state = ResGen_3_8( zk, state, params, 1/fs )
    
    where state is a structure with the state of the residual generator.
    The state has fields: p3, p2
    
    File generated Sat Jul 30 21:29:07 2016

